In [ ]:
import os,sys,time,copy
import random
import numpy as np
import pandas as pd
from scipy import interpolate
from scipy.integrate import fixed_quad,quad 
#from  numba import jit,objmode,njit
from tqdm import tqdm

from mpmath import fp

#--matplotlib
import matplotlib
from matplotlib.lines import Line2D
matplotlib.rcParams['text.latex.preamble']=r"\usepackage{amsmath}"
matplotlib.rc('text',usetex=True)
import pylab as py
from matplotlib import colors
import matplotlib.gridspec as gridspec

#--torch
import torch
import torch.nn as nn
import torch.optim as optim

print(torch.cuda.is_available())
torch.cuda.empty_cache()
manualSeed = 999 # Set random seed for reproducibility
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
#torch.use_deterministic_algorithms(True) # Needed for reproducible results
#! Number of GPUs available. Use 0 for CPU mode.
ngpu = 1 
#! Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


In [ ]:
from scipy.special import gamma
from scipy.interpolate import griddata,RegularGridInterpolator,CubicSpline

import qcdlib.params as params
import qcdlib.cfg as cfg
from qcdlib.alphaS  import ALPHAS
from qcdlib.alphaS_dev  import ALPHAS as ALPHAS_DEV
from qcdlib.eweak   import EWEAK
from qcdlib.mellin import MELLIN
import qcdlib.special as special

from qcf.qcd_qcf_1d import PDF

In [ ]:
mellin = MELLIN()
alphaS = ALPHAS()
pdf = PDF(mellin,alphaS)

In [ ]:
X = 10**np.linspace(-4,0,100)
Q2 = 10.0

uv = []
dv = []
g = []
for x in X:
    uv.append(pdf.get_xF(x,Q2,'uv'))
    dv.append(pdf.get_xF(x,Q2,'dv'))
    g.append(pdf.get_xF(x,Q2,'g'))
uv = np.array(uv)
dv = np.array(dv)
g = np.array(g)

In [ ]:
nrows,ncols=1,1
py.figure(figsize=(8*ncols,6*nrows))
ax=py.subplot(nrows,ncols,1)

ax.plot(X,uv,label=r'$u_v$')
ax.plot(X,dv,label=r'$d_v$')
ax.plot(X,g/10,label=r'$g/10$')
ax.legend(fontsize=18)
# ax.semilogx()
ax.set_xlabel(r'$x$',size=30)
ax.set_ylabel(r'$xf(x,Q^2=10)$',size=30)
ax.tick_params(direction='in',labelsize=18,which='both',axis='both')


In [ ]:
class MODEL:
    
    def __init__(self):
        #--params
        euler=special.euler

        self.Q0 = np.sqrt(cfg.Q20)
        self.C1=2*np.exp(-euler)
        self.C2=1
        
        self.bmax = self.C1/self.Q0 #--so that mub > mu_0
    
    #============================================================
    #--Defining functions associated with bstar
    #============================================================
    #--typical logarithm in perturbative expansions
    def get_Log(self,bT,mu):  
        """
        Typical log in the expansion
        """
        return np.log(mu*bT/self.C1)

    #--Setting up b_*
    def get_bstar(self,bT):
        """
        Returns b*
        """
        return bT/np.sqrt(1+bT**2/self.bmax**2)

    def get_mub(self,bT): 
        """
        Returns mub* 
        """
        return self.C1/self.get_bstar(bT)


In [ ]:
tmdmodel = MODEL()

In [ ]:
nrows,ncols = 1,2
py.figure(figsize=(8*ncols,6*nrows))

bT = 10**np.linspace(-2,1,100)

bstar = tmdmodel.get_bstar(bT)

mubstar = tmdmodel.get_mub(bT)
mu_nobstar = tmdmodel.C1 / bT

ax=py.subplot(nrows,ncols,1)
ax.plot(bT,bT,label=r'$b_T$')
ax.plot(bT,bstar,label=r'$b_*$')
ax.axvline(x=tmdmodel.bmax,ls=':',c='grey')
ax.axhline(y=tmdmodel.bmax,c='grey',label=r'$b_{\rm max}$')
ax.tick_params(direction='in',labelsize=18,axis='both',which='both')
# ax.set_xlim(0,2)
# ax.set_ylim(0,2)
ax.set_ylabel(r'$b~({\rm GeV}^{-1})$',size=30)
ax.loglog()
ax.legend(fontsize=18)
ax.set_xlabel(r'$b_T~({\rm GeV}^{-1})$',size=30)

ax=py.subplot(nrows,ncols,2)
ax.plot(bT,mu_nobstar,label=r'$\mu_b(b_T)$')
ax.plot(bT,mubstar,label=r'$\mu_b(b_*)$')
ax.axvline(x=tmdmodel.bmax,ls=':',c='grey')
ax.axhline(y= tmdmodel.C1 / tmdmodel.bmax,ls='-',c='grey',label=r'$\mu_b(b_{\rm max})$')
ax.tick_params(direction='in',labelsize=18,axis='both',which='both')
ax.semilogx()
ax.semilogy()
ax.set_xlabel(r'$b_T~({\rm GeV}^{-1})$',size=30)
ax.set_ylabel(r'$\mu_b~({\rm GeV})$',size=30)
ax.legend(fontsize=18)


In [ ]:
class PDF_OPE:
    
    def __init__(self,pdf):

        self.iorder = cfg.tmd_order #--LO: 0, NLO: 1
        
        self.pdf = pdf
        self.mellin = pdf.mellin
        
        self.tmdmodel=tmdmodel
        self.alphaS=alphaS
        
        self.CF=params.CF
        self.CA=params.CA
        self.TR=params.TR
        self.TF=params.TF
        
        #--set_charges
        quarkcharges = np.zeros(11)
        quarkcharges[0] = 0 # gluon
        quarkcharges[1] = 2/3 # u
        quarkcharges[2] =-1/3 # d
        quarkcharges[3] =-1/3 # s
        quarkcharges[4] = 2/3 # c
        quarkcharges[5] =-1/3 # b

        quarkcharges[-1] =-2/3 # baru
        quarkcharges[-2] = 1/3 # bard
        quarkcharges[-3] = 1/3 # bars
        quarkcharges[-4] =-2/3 # barc
        quarkcharges[-5] = 1/3 # barb
        self.quarkcharges=quarkcharges
        #============================================================
        #--Storages
        #============================================================

        self.storage_ope={}
        self.fmap={1:'u',2:'d',3:'s',4:'c',5:'b',-5:'bb',-4:'cb',-3:'sb',-2:'db',-1:'ub',0:'g'}
        
        self.setup()

    def reset_storage_ope(self):
        self.storage_ope={}

    #--this function is in special.get_psi(i,N)
    #def get_psi(self,i,N):
    #    return fp.psi(i,complex(N.real,N.imag))

    def setup(self):
        #============================================================
        #--Setting up OPE
        #============================================================
        N=self.mellin.N
        self.N=N
        
        psi0N = special.get_psi(0,N)
        
        self.NLOq1= -1/N - 1/(N+1) - 2*special.euler - 2*psi0N # M transform of Pqq=(2/(1-x)_{+} -1-x)
        self.NLOq2=  1/N - 1/(N+1) # M transform of (1-x)

        self.NLOg1= 1/N - 2/(N+1) + 2/(N+2)  # M transform of Pgq=1-2x(1-x)
        self.NLOg2= 1/(N+1)-1/(N+2)  # M tranform of x(1-x) 

        
    def get_OPE_TMDPDF(self,x,bT):
        """
        Returns an 11-dimensional array of flavors for the OPE for a given x and bT
        x: float
        bT: float
        """
        
        mub   = self.tmdmodel.get_mub(bT) # returns mub*
        zeta = mub**2
        bstar = self.tmdmodel.get_bstar(bT) # returns b*
        
        key=(x,bT,zeta)

        if key in self.storage_ope:
            if float(mub**2) not in self.pdf.storage:
                self.reset_storage_ope()
        
        if key not in self.storage_ope:
            
            N=self.mellin.N           

            aS=self.alphaS.get_a(mub**2) #- alphaS/(4 pi)
            
            Log=self.tmdmodel.get_Log(bstar,mub)

            self.pdf.evolve(mub**2)
            
            moments=self.pdf.storage[mub**2]

            Cq=np.ones(N.size,dtype=np.complex128)
            Cg=np.zeros(N.size,dtype=np.complex128)

            if self.iorder>=1:
                Cq+=aS*self.CF*(-4*Log**2-4*Log*np.log(zeta/mub**2)-4*Log*self.NLOq1+2*self.NLOq2-np.pi**2/6) 
                Cg+=aS*self.TF*(-4*Log*self.NLOg1+4*self.NLOg2) 

            mellin_convolutions=np.zeros((11,N.size),dtype=np.complex128)
            for i in [1,2,3,4,5]:
                mellin_convolutions[+i]=Cq*moments[self.fmap[+i]] + Cg*moments['g']
                mellin_convolutions[-i]=Cq*moments[self.fmap[-i]] + Cg*moments['g']

            xspace_convolutions=torch.zeros(11)
            for i in [1,2,3,4,5]:
                xspace_convolutions[+i]=self.mellin.invert(x,mellin_convolutions[+i])
                xspace_convolutions[-i]=self.mellin.invert(x,mellin_convolutions[-i])
            self.storage_ope[key]=xspace_convolutions

        return self.storage_ope[key]


In [ ]:
Xs = np.linspace(0.01,0.7,6)
bTs = 10**np.linspace(-2,1,100)

nrows,ncols=2,3
py.figure(figsize=(8*ncols,6*nrows))
cnt=0

flav_lab = [r'$g$',r'$u$',r'$d$',r'$s$',r'$c$',r'$b$',r'$\bar{b}$',r'$\bar{c}$',r'$\bar{s}$',r'$\bar{d}$',r'$\bar{u}$']
for x in Xs:
    cnt+=1
    ax=py.subplot(nrows,ncols,cnt)

    opes = np.zeros((11,len(bTs)))
    for i in range(len(bTs)):
        opetmd = ope.get_OPE_TMDPDF(x,bTs[i])
        for j in range(len(opes)):
            opes[j][i] = opetmd[j]

    for i in range(len(opes)):
        if i in [0,4,5,6,7,8,9,10]: continue
        ax.plot(bTs, opes[i],label=flav_lab[i])

    ax.semilogx()
    ax.set_xlabel(r'$b_T~({\rm GeV}^{-1})$',size=30)
    ax.set_ylabel(r'$\tilde{f}_{\rm OPE}(x,b_T)$',size=30)
    ax.text(0.1,0.9,r'$x=%.2f$'%x,transform=ax.transAxes,size=30)
    if cnt==1: ax.legend(fontsize=18)
    ax.tick_params(direction='in',labelsize=18,which='both',axis='both')